Importing the libraries and reading the CSV files

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from math import pi
import seaborn as sns
from scipy.ndimage.filters import gaussian_filter1d
from datetime import datetime
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [ ]:
# The two variables below needs to be changed once a new version of the dataset will be published or once different line charts are required
end_date = '2020-10-31'
smoothing_factor = 2

In [ ]:
df16_players = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin16_players.csv", low_memory=False)
df17_players = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin17_players.csv", low_memory=False)
df18_players = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin18_players.csv", low_memory=False)
df19_players = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin19_players.csv", low_memory=False)
df20_players = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin20_players.csv", low_memory=False)
df21_players = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin21_players.csv", low_memory=False)

df16_prices = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin16_prices.csv", low_memory=False)
df17_prices = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin17_prices.csv", low_memory=False)
df18_prices = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin18_prices.csv", low_memory=False)
df19_prices = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin19_prices.csv", low_memory=False)
df20_prices = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin20_prices.csv", low_memory=False)
df21_prices = pd.read_csv("../input/fifa-21-ultimate-team-players-and-prices-dataset/fut_bin21_prices.csv", low_memory=False)

df16_prices['date'] = pd.to_datetime(df16_prices['date'])
df17_prices['date'] = pd.to_datetime(df17_prices['date'])
df18_prices['date'] = pd.to_datetime(df18_prices['date'])
df19_prices['date'] = pd.to_datetime(df19_prices['date'])
df20_prices['date'] = pd.to_datetime(df20_prices['date'])
df21_prices['date'] = pd.to_datetime(df21_prices['date'])
df16_prices[['ps4', 'xbox']] = df16_prices[['ps4', 'xbox']].apply(pd.to_numeric, errors='coerce')
df17_prices[['ps4', 'xbox']] = df17_prices[['ps4', 'xbox']].apply(pd.to_numeric, errors='coerce')
df18_prices[['ps4', 'xbox']] = df18_prices[['ps4', 'xbox']].apply(pd.to_numeric, errors='coerce')
df19_prices[['ps4', 'xbox']] = df19_prices[['ps4', 'xbox']].apply(pd.to_numeric, errors='coerce')
df20_prices[['ps4', 'xbox']] = df20_prices[['ps4', 'xbox']].apply(pd.to_numeric, errors='coerce')
df21_prices[['ps4', 'xbox']] = df21_prices[['ps4', 'xbox']].apply(pd.to_numeric, errors='coerce')

# in FIFA 16 De Bruyne had a different resource_id
df16_players.loc[df16_players['resource_id'] == 50524633, 'resource_id'] = 192985
df16_prices.loc[df16_prices['resource_id'] == 50524633, 'resource_id'] = 192985

# in FIFA 16 Martial had a different resource_id
df16_players.loc[df16_players['resource_id'] == 50542948, 'resource_id'] = 211300
df16_prices.loc[df16_prices['resource_id'] == 50542948, 'resource_id'] = 211300

In [ ]:
df21_players.head()

In [ ]:
df21_prices.head()

In [ ]:
column_names = {'pace': 'Pace', 'dribbling': 'Dribble', 'shooting': 'Shoot',
                'passing': 'Pass', 'defending': 'Defend', 'physicality': 'Physic',
                'pace_acceleration': 'Accel', 'pace_sprint_speed': 'Sprint', 'phys_jumping': 'Jump',
                'phys_stamina': 'Stamina', 'phys_strength': 'Strength', 'phys_aggression': 'Aggress',
                'drib_agility': 'Agility', 'drib_balance': 'Balance', 'drib_reactions': 'React',
                'drib_ball_control': 'Ball ctrl', 'drib_dribbling': 'Dribbl', 'drib_composure': 'Compos',
                'shoot_positioning': 'Posit', 'shoot_finishing': 'Finish', 'shoot_shot_power': 'Shot power',
                'shoot_long_shots': 'Long shot', 'shoot_volleys': 'Volleys', 'shoot_penalties': 'Penalty'}
attribute_columns_filter = ['Pace', 'Dribble', 'Shoot', 'Pass', 'Defend', 'Physic']
physical_columns_filter = ['Accel', 'Sprint', 'Jump', 'Stamina', 'Strength', 'Aggress']
dribbling_columns_filter = ['Agility', 'Balance', 'React', 'Ball ctrl', 'Dribbl', 'Compos']
shooting_columns_filter = ['Posit', 'Finish', 'Shot power', 'Long shot', 'Volleys', 'Penalty']


def filtering_players_df(unfiltered_df):
    filtered_df = unfiltered_df[unfiltered_df['resource_id'].isin(resource_ids.values())].sort_values(by='player_name', ascending=True).set_index('player_name')
    filtered_df.rename(columns=column_names, inplace=True)
    attributes_df = filtered_df[attribute_columns_filter]
    physical_df = filtered_df[physical_columns_filter]
    dribbling_df = filtered_df[dribbling_columns_filter]
    shooting_df = filtered_df[shooting_columns_filter]
    return attributes_df, physical_df, dribbling_df, shooting_df


def concatenating_six_prices_df(field_name, df1, df2, df3, df4, df5, df6):
    merged_df = pd.concat([
        df1[(df1['resource_id'].isin(resource_ids.values())) & (df1['date'].between('2015-10-15', '2016-08-15'))][['resource_id' , 'date', field_name]].set_index('date'),
        df2[(df2['resource_id'].isin(resource_ids.values())) & (df2['date'].between('2016-10-15', '2017-08-15'))][['resource_id' , 'date', field_name]].set_index('date'),
        df3[(df3['resource_id'].isin(resource_ids.values())) & (df3['date'].between('2017-10-15', '2018-08-15'))][['resource_id' , 'date', field_name]].set_index('date'),
        df4[(df4['resource_id'].isin(resource_ids.values())) & (df4['date'].between('2018-10-15', '2019-08-15'))][['resource_id' , 'date', field_name]].set_index('date'),
        df5[(df5['resource_id'].isin(resource_ids.values())) & (df5['date'].between('2019-10-15', '2020-08-15'))][['resource_id' , 'date', field_name]].set_index('date'),
        df6[(df6['resource_id'].isin(resource_ids.values())) & (df6['date'].between('2020-10-15', end_date))][['resource_id' , 'date', field_name]].set_index('date')])
    if len(list(resource_ids.keys())) == 2:
        df_prices_1 = merged_df[merged_df['resource_id'] == list(resource_ids.values())[0]].drop('resource_id', axis=1).rename(columns = {field_name: list(resource_ids.keys())[0]})
        df_prices_2 = merged_df[merged_df['resource_id'] == list(resource_ids.values())[1]].drop('resource_id', axis=1).rename(columns = {field_name: list(resource_ids.keys())[1]})
        clean_df = pd.concat([df_prices_1, df_prices_2], axis=1)
    elif len(list(resource_ids.keys())) == 3:
        df_prices_1 = merged_df[merged_df['resource_id'] == list(resource_ids.values())[0]].drop('resource_id', axis=1).rename(columns = {field_name: list(resource_ids.keys())[0]})
        df_prices_2 = merged_df[merged_df['resource_id'] == list(resource_ids.values())[1]].drop('resource_id', axis=1).rename(columns = {field_name: list(resource_ids.keys())[1]})
        df_prices_3 = merged_df[merged_df['resource_id'] == list(resource_ids.values())[2]].drop('resource_id', axis=1).rename(columns = {field_name: list(resource_ids.keys())[2]})
        clean_df = pd.concat([df_prices_1, df_prices_2, df_prices_3], axis=1)
    clean_df.reset_index(inplace=True)
    return clean_df


def subplot_two_players_comparison(input_df):
    attributes_data = list(input_df)
    att_no = len(attributes_data)
    angles_data = [n / float(att_no) * 2 * pi for n in range(att_no)] + [n / float(att_no) * 2 * pi for n in range(att_no)][:1]
    first_values = input_df.iloc[0].tolist() + input_df.iloc[0].tolist()[:1]
    second_values = input_df.iloc[1].tolist() + input_df.iloc[1].tolist()[:1]
    return attributes_data, angles_data, first_values, second_values

def subplot_three_players_comparison(input_df):
    attributes_data = list(input_df)
    att_no = len(attributes_data)
    angles_data = [n / float(att_no) * 2 * pi for n in range(att_no)] + [n / float(att_no) * 2 * pi for n in range(att_no)][:1]
    first_values = input_df.iloc[0].tolist() + input_df.iloc[0].tolist()[:1]
    second_values = input_df.iloc[1].tolist() + input_df.iloc[1].tolist()[:1]
    third_values = input_df.iloc[2].tolist() + input_df.iloc[2].tolist()[:1]
    return attributes_data, angles_data, first_values, second_values, third_values


def radar_graphs_comparison():
    if len(resource_ids.keys()) == 2:
        plt.figure(figsize=(20, 12))
        # 1st subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df17_attributes)
        ax = plt.subplot(4, 5, 1, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        plt.figtext(0.1, 0.98, first_player, color="C0", size=12)
        plt.figtext(0.1, 0.95, second_player, color="C1", size=12)
        plt.title('FIFA 17', size=11, y=1.1)
        # 2nd subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df18_attributes)
        ax = plt.subplot(4, 5, 2, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        plt.title('FIFA 18', size=11, y=1.1)
        # 3rd subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df19_attributes)
        ax = plt.subplot(4, 5, 3, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        plt.title('FIFA 19', size=11, y=1.1)
        # 4th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df20_attributes)
        ax = plt.subplot(4, 5, 4, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        plt.title('FIFA 20', size=11, y=1.1)
        # 5th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df21_attributes)
        ax = plt.subplot(4, 5, 5, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        plt.title('FIFA 21', size=11, y=1.1)
        # 6th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df17_physical)
        ax = plt.subplot(4, 5, 6, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 7th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df18_physical)
        ax = plt.subplot(4, 5, 7, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 8th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df19_physical)
        ax = plt.subplot(4, 5, 8, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 9th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df20_physical)
        ax = plt.subplot(4, 5, 9, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 10th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df21_physical)
        ax = plt.subplot(4, 5, 10, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 11th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df17_dribbling)
        ax = plt.subplot(4, 5, 11, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 12th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df18_dribbling)
        ax = plt.subplot(4, 5, 12, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 13th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df19_dribbling)
        ax = plt.subplot(4, 5, 13, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 14th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df20_dribbling)
        ax = plt.subplot(4, 5, 14, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 15th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df21_dribbling)
        ax = plt.subplot(4, 5, 15, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 16th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df17_shooting)
        ax = plt.subplot(4, 5, 16, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 17th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df18_shooting)
        ax = plt.subplot(4, 5, 17, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 18th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df19_shooting)
        ax = plt.subplot(4, 5, 18, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 19th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df20_shooting)
        ax = plt.subplot(4, 5, 19, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        # 20th subplot
        attributes, angles, values1, values2 = subplot_two_players_comparison(df21_shooting)
        ax = plt.subplot(4, 5, 20, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
    elif len(resource_ids.keys()) == 3:
        plt.figure(figsize=(20, 12))
        # 1st subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df17_attributes)
        ax = plt.subplot(4, 5, 1, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        plt.figtext(0.1, 0.98, first_player, color="C0", size=12)
        plt.figtext(0.1, 0.95, second_player, color="C1", size=12)
        plt.figtext(0.1, 0.92, third_player, color="C2", size=12)
        plt.title('FIFA 17', size=11, y=1.1)
        # 2nd subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df18_attributes)
        ax = plt.subplot(4, 5, 2, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        plt.title('FIFA 18', size=11, y=1.1)
        # 3rd subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df19_attributes)
        ax = plt.subplot(4, 5, 3, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        plt.title('FIFA 19', size=11, y=1.1)
        # 4th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df20_attributes)
        ax = plt.subplot(4, 5, 4, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        plt.title('FIFA 20', size=11, y=1.1)
        # 5th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df21_attributes)
        ax = plt.subplot(4, 5, 5, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        plt.title('FIFA 21', size=11, y=1.1)
        # 6th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df17_physical)
        ax = plt.subplot(4, 5, 6, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 7th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df18_physical)
        ax = plt.subplot(4, 5, 7, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 8th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df19_physical)
        ax = plt.subplot(4, 5, 8, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 9th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df20_physical)
        ax = plt.subplot(4, 5, 9, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 10th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df21_physical)
        ax = plt.subplot(4, 5, 10, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 11th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df17_dribbling)
        ax = plt.subplot(4, 5, 11, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 12th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df18_dribbling)
        ax = plt.subplot(4, 5, 12, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 13th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df19_dribbling)
        ax = plt.subplot(4, 5, 13, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 14th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df20_dribbling)
        ax = plt.subplot(4, 5, 14, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 15th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df21_dribbling)
        ax = plt.subplot(4, 5, 15, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 16th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df17_shooting)
        ax = plt.subplot(4, 5, 16, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 17th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df18_shooting)
        ax = plt.subplot(4, 5, 17, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 18th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df19_shooting)
        ax = plt.subplot(4, 5, 18, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 19th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df20_shooting)
        ax = plt.subplot(4, 5, 19, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)
        # 20th subplot
        attributes, angles, values1, values2, values3 = subplot_three_players_comparison(df21_shooting)
        ax = plt.subplot(4, 5, 20, polar=True)
        plt.xticks(angles[:-1], attributes, color='grey', size=10)
        plt.yticks([25,50,75], ["25","50","75"], color="grey", size=8)
        plt.ylim(0,100)
        plt.subplots_adjust(wspace=0.35)
        ax.plot(angles, values1, linewidth=1, linestyle='solid')
        ax.fill(angles, values1, 'C0', alpha=0.1)
        ax.plot(angles, values2, linewidth=1, linestyle='solid')
        ax.fill(angles, values2, 'C1', alpha=0.1)
        ax.plot(angles, values3, linewidth=1, linestyle='solid')
        ax.fill(angles, values3, 'C2', alpha=0.1)


def prices_comparison(platform_name, prices_df):
    if platform_name == 'ps4':
        if len(resource_ids.keys()) == 2:
            sns.set()
            sns.set_context("talk")
            # line plot
            plt.figure(figsize=(25, 10))
            ax = plt.gca()
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[first_player], sigma=smoothing_factor), label=first_player, color='C0')
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[second_player], sigma=smoothing_factor), label=second_player, color='C1')
            ax.set_ylabel('Price')
            ax.set_xlim([datetime(2015, 9, 30), datetime.strptime(end_date, "%Y-%m-%d")])
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            ax.legend()
            ax.set_title('FIFA 16-21 - ' + first_player + ' vs ' + second_player + ' prices on PS4')
            plt.gcf().autofmt_xdate()
            plt.show()
        elif len(resource_ids.keys()) == 3:
            sns.set()
            sns.set_context("talk")
            # line plot
            plt.figure(figsize=(25, 10))
            ax = plt.gca()
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[first_player], sigma=smoothing_factor), label=first_player, color='C0')
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[second_player], sigma=smoothing_factor), label=second_player, color='C1')
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[third_player], sigma=smoothing_factor), label=third_player, color='C2')
            ax.set_ylabel('Price')
            ax.set_xlim([datetime(2015, 9, 30), datetime.strptime(end_date, "%Y-%m-%d")])
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            ax.legend()
            ax.set_title('FIFA 16-21 - ' + first_player + ' vs ' + second_player + ' vs ' + third_player + ' prices on PS4')
            plt.gcf().autofmt_xdate()
            plt.show()
    elif platform_name == 'xbox':
        if len(resource_ids.keys()) == 2:
            sns.set()
            sns.set_context("talk")
            # line plot
            plt.figure(figsize=(25, 10))
            ax = plt.gca()
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[first_player], sigma=smoothing_factor), label=first_player, color='C9')
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[second_player], sigma=smoothing_factor), label=second_player, color='C8')
            ax.set_ylabel('Price')
            ax.set_xlim([datetime(2015, 9, 30), datetime.strptime(end_date, "%Y-%m-%d")])
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            ax.legend()
            ax.set_title('FIFA 16-21 - ' + first_player + ' vs ' + second_player + ' prices on Xbox')
            plt.gcf().autofmt_xdate()
            plt.show()
        elif len(resource_ids.keys()) == 3:
            sns.set()
            sns.set_context("talk")
            # line plot
            plt.figure(figsize=(25, 10))
            ax = plt.gca()
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[first_player], sigma=smoothing_factor), label=first_player, color='C9')
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[second_player], sigma=smoothing_factor), label=second_player, color='C8')
            ax.plot(prices_df['date'], gaussian_filter1d(prices_df[third_player], sigma=smoothing_factor), label=third_player, color='C7')
            ax.set_ylabel('Price')
            ax.set_xlim([datetime(2015, 9, 30), datetime.strptime(end_date, "%Y-%m-%d")])
            ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
            ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
            ax.legend()
            ax.set_title('FIFA 16-21 - ' + first_player + ' vs ' + second_player + ' vs ' + third_player + ' prices on Xbox')
            plt.gcf().autofmt_xdate()
            plt.show()


def price_ratios_comparison(ps4_df, xbox_df):
    if len(resource_ids.keys()) == 2:
        ps4_df.columns = ['date', first_player + ' - PS4', second_player + ' - PS4']
        xbox_df.columns = ['date', first_player + ' - Xbox', second_player + ' - Xbox']
        merged_prices_df = ps4_df.merge(xbox_df, on='date')
        merged_prices_df[first_player] = ((merged_prices_df[first_player + ' - PS4'] / merged_prices_df[first_player + ' - Xbox']) - 1) * 100
        merged_prices_df[second_player] = ((merged_prices_df[second_player + ' - PS4'] / merged_prices_df[second_player + ' - Xbox']) - 1) * 100
        merged_prices_df = merged_prices_df[['date', first_player, second_player]]
        # configuring seaborn
        sns.set()
        sns.set_context("talk")
        # line plot
        plt.figure(figsize=(25, 10))
        ax = plt.gca()
        ax.plot(merged_prices_df['date'], gaussian_filter1d(merged_prices_df[first_player], sigma=smoothing_factor), label=first_player, color='C4')
        ax.plot(merged_prices_df['date'], gaussian_filter1d(merged_prices_df[second_player], sigma=smoothing_factor), label=second_player, color='C2')
        ax.set_ylabel('Δ PS4 vs Xbox price (in %)')
        ax.set_xlim([datetime(2015, 9, 30), datetime.strptime(end_date, "%Y-%m-%d")])
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.legend()
        ax.set_title('FIFA 16-21 - ' + first_player + ' vs ' + second_player + ' ratio between PS4 and Xbox prices')
        plt.gcf().autofmt_xdate()
        plt.show()
    elif len(resource_ids.keys()) == 3:
        ps4_df.columns = ['date', first_player + ' - PS4', second_player + ' - PS4', third_player + ' - PS4']
        xbox_df.columns = ['date', first_player + ' - Xbox', second_player + ' - Xbox', third_player + ' - Xbox']
        merged_prices_df = ps4_df.merge(xbox_df, on='date')
        merged_prices_df[first_player] = ((merged_prices_df[first_player + ' - PS4'] / merged_prices_df[first_player + ' - Xbox']) - 1) * 100
        merged_prices_df[second_player] = ((merged_prices_df[second_player + ' - PS4'] / merged_prices_df[second_player + ' - Xbox']) - 1) * 100
        merged_prices_df[third_player] = ((merged_prices_df[third_player + ' - PS4'] / merged_prices_df[third_player + ' - Xbox']) - 1) * 100
        merged_prices_df = merged_prices_df[['date', first_player, second_player, third_player]]
        # configuring seaborn
        sns.set()
        sns.set_context("talk")
        # line plot
        plt.figure(figsize=(25, 10))
        ax = plt.gca()
        ax.plot(merged_prices_df['date'], gaussian_filter1d(merged_prices_df[first_player], sigma=smoothing_factor), label=first_player, color='C4')
        ax.plot(merged_prices_df['date'], gaussian_filter1d(merged_prices_df[second_player], sigma=smoothing_factor), label=second_player, color='C2')
        ax.plot(merged_prices_df['date'], gaussian_filter1d(merged_prices_df[third_player], sigma=smoothing_factor), label=third_player, color='C6')
        ax.set_ylabel('Δ PS4 vs Xbox price (in %)')
        ax.set_xlim([datetime(2015, 9, 30), datetime.strptime(end_date, "%Y-%m-%d")])
        ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
        ax.legend()
        ax.set_title('FIFA 16-21 - ' + first_player + ' vs ' + second_player + ' vs ' + third_player + ' ratio between PS4 and Xbox prices')
        plt.gcf().autofmt_xdate()
        plt.show()

**A classic players comparison to warm up - Messi vs Ronaldo**

In [ ]:
resource_ids = {'Messi': 158023,
                'Ronaldo': 20801}

df17_attributes, df17_physical, df17_dribbling, df17_shooting = filtering_players_df(df17_players)
df18_attributes, df18_physical, df18_dribbling, df18_shooting = filtering_players_df(df18_players)
df19_attributes, df19_physical, df19_dribbling, df19_shooting = filtering_players_df(df19_players)
df20_attributes, df20_physical, df20_dribbling, df20_shooting = filtering_players_df(df20_players)
df21_attributes, df21_physical, df21_dribbling, df21_shooting = filtering_players_df(df21_players)

if len(resource_ids.keys()) == 2:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
elif len(resource_ids.keys()) == 3:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
    third_player = df21_attributes.index[2]

df_prices_ps4 = concatenating_six_prices_df('ps4', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)
df_prices_xbox = concatenating_six_prices_df('xbox', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)

In [ ]:
radar_graphs_comparison()

In [ ]:
prices_comparison('ps4', df_prices_ps4)

In [ ]:
prices_comparison('xbox', df_prices_xbox)

In [ ]:
price_ratios_comparison(df_prices_ps4, df_prices_xbox)

**Liverpool current front 3 - Firmino vs Mane vs Salah**

In [ ]:
resource_ids = {'Firmino': 201942,
                'Mané': 208722,
                'Salah': 209331}

df17_attributes, df17_physical, df17_dribbling, df17_shooting = filtering_players_df(df17_players)
df18_attributes, df18_physical, df18_dribbling, df18_shooting = filtering_players_df(df18_players)
df19_attributes, df19_physical, df19_dribbling, df19_shooting = filtering_players_df(df19_players)
df20_attributes, df20_physical, df20_dribbling, df20_shooting = filtering_players_df(df20_players)
df21_attributes, df21_physical, df21_dribbling, df21_shooting = filtering_players_df(df21_players)

if len(resource_ids.keys()) == 2:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
elif len(resource_ids.keys()) == 3:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
    third_player = df21_attributes.index[2]

df_prices_ps4 = concatenating_six_prices_df('ps4', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)
df_prices_xbox = concatenating_six_prices_df('xbox', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)

In [ ]:
radar_graphs_comparison()

In [ ]:
prices_comparison('ps4', df_prices_ps4)

In [ ]:
prices_comparison('xbox', df_prices_xbox)

In [ ]:
price_ratios_comparison(df_prices_ps4, df_prices_xbox)

**Man City current front 3 - Agüero vs De Bruyne vs Sterling**

In [ ]:
resource_ids = {'Agüero': 153079,
                'De Bruyne': 192985,
                'Sterling': 202652}

df17_attributes, df17_physical, df17_dribbling, df17_shooting = filtering_players_df(df17_players)
df18_attributes, df18_physical, df18_dribbling, df18_shooting = filtering_players_df(df18_players)
df19_attributes, df19_physical, df19_dribbling, df19_shooting = filtering_players_df(df19_players)
df20_attributes, df20_physical, df20_dribbling, df20_shooting = filtering_players_df(df20_players)
df21_attributes, df21_physical, df21_dribbling, df21_shooting = filtering_players_df(df21_players)

if len(resource_ids.keys()) == 2:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
elif len(resource_ids.keys()) == 3:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
    third_player = df21_attributes.index[2]

df_prices_ps4 = concatenating_six_prices_df('ps4', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)
df_prices_xbox = concatenating_six_prices_df('xbox', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)

In [ ]:
radar_graphs_comparison()

In [ ]:
prices_comparison('ps4', df_prices_ps4)

In [ ]:
prices_comparison('xbox', df_prices_xbox)

In [ ]:
price_ratios_comparison(df_prices_ps4, df_prices_xbox)

**Man Utd current front 3 - Fernandes vs Martial vs Rashford**

In [ ]:
resource_ids = {'Fernandes': 212198,
                'Rashford': 231677,
                'Martial': 211300}

df17_attributes, df17_physical, df17_dribbling, df17_shooting = filtering_players_df(df17_players)
df18_attributes, df18_physical, df18_dribbling, df18_shooting = filtering_players_df(df18_players)
df19_attributes, df19_physical, df19_dribbling, df19_shooting = filtering_players_df(df19_players)
df20_attributes, df20_physical, df20_dribbling, df20_shooting = filtering_players_df(df20_players)
df21_attributes, df21_physical, df21_dribbling, df21_shooting = filtering_players_df(df21_players)

if len(resource_ids.keys()) == 2:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
elif len(resource_ids.keys()) == 3:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
    third_player = df21_attributes.index[2]

df_prices_ps4 = concatenating_six_prices_df('ps4', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)
df_prices_xbox = concatenating_six_prices_df('xbox', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)

In [ ]:
radar_graphs_comparison()

In [ ]:
prices_comparison('ps4', df_prices_ps4)

In [ ]:
prices_comparison('xbox', df_prices_xbox)

In [ ]:
price_ratios_comparison(df_prices_ps4, df_prices_xbox)

**Chelsea current front 3 - Pulisic vs Werner vs Ziyech**

In [ ]:
resource_ids = {'Pulisic': 227796,
                'Werner': 212188,
                'Ziyech': 208670}

df17_attributes, df17_physical, df17_dribbling, df17_shooting = filtering_players_df(df17_players)
df18_attributes, df18_physical, df18_dribbling, df18_shooting = filtering_players_df(df18_players)
df19_attributes, df19_physical, df19_dribbling, df19_shooting = filtering_players_df(df19_players)
df20_attributes, df20_physical, df20_dribbling, df20_shooting = filtering_players_df(df20_players)
df21_attributes, df21_physical, df21_dribbling, df21_shooting = filtering_players_df(df21_players)

if len(resource_ids.keys()) == 2:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
elif len(resource_ids.keys()) == 3:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
    third_player = df21_attributes.index[2]

df_prices_ps4 = concatenating_six_prices_df('ps4', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)
df_prices_xbox = concatenating_six_prices_df('xbox', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)

In [ ]:
radar_graphs_comparison()

In [ ]:
prices_comparison('ps4', df_prices_ps4)

In [ ]:
prices_comparison('xbox', df_prices_xbox)

In [ ]:
price_ratios_comparison(df_prices_ps4, df_prices_xbox)

**Tottenham current front 3 - Bale vs Kane vs Son**

In [ ]:
resource_ids = {'Bale': 173731,
                'Kane': 202126,
                'Son': 200104}

df17_attributes, df17_physical, df17_dribbling, df17_shooting = filtering_players_df(df17_players)
df18_attributes, df18_physical, df18_dribbling, df18_shooting = filtering_players_df(df18_players)
df19_attributes, df19_physical, df19_dribbling, df19_shooting = filtering_players_df(df19_players)
df20_attributes, df20_physical, df20_dribbling, df20_shooting = filtering_players_df(df20_players)
df21_attributes, df21_physical, df21_dribbling, df21_shooting = filtering_players_df(df21_players)

if len(resource_ids.keys()) == 2:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
elif len(resource_ids.keys()) == 3:
    first_player = df21_attributes.index[0]
    second_player = df21_attributes.index[1]
    third_player = df21_attributes.index[2]

df_prices_ps4 = concatenating_six_prices_df('ps4', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)
df_prices_xbox = concatenating_six_prices_df('xbox', df16_prices, df17_prices, df18_prices, df19_prices, df20_prices, df21_prices)

In [ ]:
radar_graphs_comparison()

In [ ]:
prices_comparison('ps4', df_prices_ps4)

In [ ]:
prices_comparison('xbox', df_prices_xbox)

In [ ]:
price_ratios_comparison(df_prices_ps4, df_prices_xbox)